In [33]:
from __future__ import division
from astropy.io import fits
import numpy as np
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs
from pylab import *
from numpy import *
import scipy
import matplotlib.pyplot as pyplot
import scipy.integrate
from scipy.integrate import quad
from scipy.interpolate import UnivariateSpline
from matplotlib import gridspec
import scipy.optimize
from scipy.optimize import minimize
import scipy.stats
import matplotlib.ticker as ticker
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from matplotlib import rc
from astropy.visualization import simple_norm
from astropy.table import Table
from scipy.optimize import curve_fit
from matplotlib.ticker import EngFormatter, StrMethodFormatter
import csv
from astropy import wcs
from astropy.io import fits
import datetime
import sys
import statistics
from scipy import ndimage, misc
import scipy.optimize as opt
from scipy.ndimage.measurements import label
from scipy.special import kv
from scipy.special import iv
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import sqlutilpy
import healpy as hp
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  
from astropy.coordinates import Angle, Latitude, Longitude 
import matplotlib.patches as patches
import sfdmap
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
from matplotlib.path import Path
import timeit
from astropy import modeling
from scipy.optimize import curve_fit
from scipy.special import wofz
# rc('font',**{'family':'serif','serif':['Computer Modern Roman']}) #latex font if i want it
# rc('text', usetex=True)
import astropy.units as u
from sklearn.neighbors import NearestNeighbors

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

    
sys.meta_path.append(NotebookFinder())

import Cke_tools as cke 

Cke_tools.ipynb


In [34]:
RRL_path = '/data/cke23/RRL_data/' # location of files 

In [35]:
nside = 64 
print("Approximate resolution at NSIDE {} is {:.2} deg".format(nside, hp.nside2resol(nside, arcmin=True) / 60))

npix = hp.nside2npix(nside) 
print('number of pixels = {}'.format(npix))

Approximate resolution at NSIDE 64 is 0.92 deg
number of pixels = 49152


In [36]:
#catalog of RRL stars
dist_min = 30
dist_cut_stars_indexed = 'nside_'+str(nside)+'_>'+str(dist_min)+'kpc_RRL_dist.fits'

hdul_data = fits.open(RRL_path+dist_cut_stars_indexed) 
data = hdul_data[1].data 
data_header = hdul_data[1].header 
data_cols = hdul_data[1].columns 
hdul_data.close()

RRL_dist_kpc = data.field('DIST') 
RRL_ra = data.field('RA') 
RRL_dec = data.field('DEC')
RRL_pmra = data.field('PMRA') 
RRL_pmdec = data.field('PMDEC') 
RRL_pmra_err = data.field('PMRA_ERROR') 
RRL_pmdec_err = data.field('PMDEC_ERROR') 
RRL_ebv = data.field('EBV') 
RRL_pixels = data.field('PIXEL')

In [57]:
#construct 5d space
RRL_coords_icrs = SkyCoord(ra=RRL_ra*u.degree, dec=RRL_dec*u.degree, pm_ra_cosdec=RRL_pmra*u.mas/u.yr, pm_dec=RRL_pmdec*u.mas/u.yr, distance=RRL_dist_kpc*u.kpc, frame='icrs')
RRL_coords_gal = RRL_coords_icrs.galactic

RRL_l, RRL_b = RRL_coords_gal.l.deg, RRL_coords_gal.b.deg
pm_l_cosb, pm_b = RRL_coords_gal.pm_l_cosb, RRL_coords_gal.pm_b #mas/yr
v_l, v_b = 4.74*np.array(pm_l_cosb)*RRL_dist_kpc, 4.74*np.array(pm_b)*RRL_dist_kpc #km/s

std_l, std_b, std_d = np.std(RRL_l), np.std(RRL_b), np.std(RRL_dist_kpc)
std_vl, std_vb = np.std(v_l), np.std(v_b)

norm_l, norm_b, norm_d = RRL_l/std_l, RRL_b/std_b, RRL_dist_kpc/std_d
norm_vl, norm_vb = v_l/std_vl, v_b/std_vb

data_5d = (np.vstack((norm_l, norm_b, norm_d, norm_vl, norm_vb))).T #format [l, b, d, vl, vb] in each row

In [62]:
neighbors = NearestNeighbors(n_neighbors=5).fit(data_5d)
distances, indices = neighbors.kneighbors(data_5d)
print(distances, indices)

[[0.         0.3558398  0.36223256 0.36732447 0.40427151]
 [0.         0.08182692 0.09269772 0.09367869 0.09413653]
 [0.         0.08918225 0.10838834 0.11628204 0.12191736]
 ...
 [0.         0.24927624 0.28885884 0.3021075  0.33030071]
 [0.         0.08146208 0.09443416 0.11422454 0.12735971]
 [0.         0.0904254  0.09159693 0.11114179 0.11133171]] [[    0 35119 57103  8671 29224]
 [    1 39012 50854 56031 55688]
 [    2 27273  7274 48892 21418]
 ...
 [65512 26232 62073  5444 51441]
 [65513 62417   940  2167 54610]
 [65514  6189 38396  6914 50453]]


In [32]:
# #calculate l, b, v_l, v_b

# RRL_coords_icrs = SkyCoord(ra=RRL_ra*u.degree, dec=RRL_dec*u.degree, pm_ra_cosdec=RRL_pmra*u.mas/u.yr, pm_dec=RRL_pmdec*u.mas/u.yr, distance=RRL_dist_kpc*u.kpc, frame='icrs')
# RRL_coords_gal = RRL_coords_icrs.galactic
# RRL_coords_cart = RRL_coords_icrs.cartesian

# RRL_l, RRL_b = RRL_coords_gal.l.deg, RRL_coords_gal.b.deg
# RRL_x, RRL_y, RRL_z = RRL_coords_cart.x, RRL_coords_cart.y, RRL_coords_cart.z

# pm_l_cosb, pm_b = RRL_coords_gal.pm_l_cosb, RRL_coords_gal.pm_b #mas/yr

# #convert proper motions to velocity
# v_l, v_b = 4.74*np.array(pm_l_cosb)*RRL_dist_kpc, 4.74*np.array(pm_b)*RRL_dist_kpc #km/s

# gal_vel=RRL_coords_gal.velocity
# icrs_vel=RRL_coords_icrs.velocity

# #which is the right velocity, what coordinate frame should I be using, how to correct for relfex of sun
# #just use my calculated v_l, v_b for now with l, b and d

# #normalise by standard deviations to remove units
# stdev_l, stdev_b, stdev_d = np.std(RRL_l), np.std(RRL_b), np.std(RRL_dist_kpc)
# stdev_vl, stdev_vb = np.std(v_l), np.std(v_b)



-51.0936308876771
21.530726386941318
-0.22178758675649643
[-0.20620661 -0.45191463  0.32431621 ... -1.78462029 -0.16285119
 -0.69191256]
